In [1]:
# Loading libraries
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score,f1_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score


In [2]:
# Loading dataset

dataset = pd.read_csv('../input/creditcard.csv')
dataset = dataset.drop(['V28','V27','V26','V25','V24','V23','V22','V20','V15','V13','V8'], axis =1)
dataset['V1_'] = dataset.V1.map(lambda x: 1 if x < -3 else 0)
dataset['V2_'] = dataset.V2.map(lambda x: 1 if x > 2.5 else 0)
dataset['V3_'] = dataset.V3.map(lambda x: 1 if x < -4 else 0)
dataset['V4_'] = dataset.V4.map(lambda x: 1 if x > 2.5 else 0)
dataset['V5_'] = dataset.V5.map(lambda x: 1 if x < -4.5 else 0)
dataset['V6_'] = dataset.V6.map(lambda x: 1 if x < -2.5 else 0)
dataset['V7_'] = dataset.V7.map(lambda x: 1 if x < -3 else 0)
dataset['V9_'] = dataset.V9.map(lambda x: 1 if x < -2 else 0)
dataset['V10_'] = dataset.V10.map(lambda x: 1 if x < -2.5 else 0)
dataset['V11_'] = dataset.V11.map(lambda x: 1 if x > 2 else 0)
dataset['V12_'] = dataset.V12.map(lambda x: 1 if x < -2 else 0)
dataset['V14_'] = dataset.V14.map(lambda x: 1 if x < -2.5 else 0)
dataset['V16_'] = dataset.V16.map(lambda x: 1 if x < -2 else 0)
dataset['V17_'] = dataset.V17.map(lambda x: 1 if x < -2 else 0)
dataset['V18_'] = dataset.V18.map(lambda x: 1 if x < -2 else 0)
dataset['V19_'] = dataset.V19.map(lambda x: 1 if x > 1.5 else 0)
dataset['V21_'] = dataset.V21.map(lambda x: 1 if x > 0.6 else 0)

#Crea una nueva función para transacciones normales (no fraudulentas).
dataset.loc[dataset.Class == 0, 'Clase'] = 0
dataset.loc[dataset.Class == 1, 'Clase'] = 1
dataset = dataset.drop(['Class'], axis =1)

In [3]:
# Parameters and results
# Disminuya todas las características que tienen distribuciones muy similares entre los dos tipos de transacciones.
print(dataset.shape[1])
n_len=dataset.shape[1]
pd.set_option("display.max_columns",101)
dataset.head()

x = dataset.iloc[: , 1:n_len-1].values
y = dataset.iloc[:, n_len-1].values

37


In [4]:
# Handling Missing Values
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer = imputer.fit(x[:, 1::n_len-1])
x[:, 1::n_len-1] = imputer.fit_transform(x[:, 1::n_len-1])

# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
x = sc_x.fit_transform(x)

#FOLDS

In [5]:
#from sklearn.model_selection import KFold, cross_validate
#scoring = ['accuracy', 'f1']
#kfold = KFold(n_splits=3)
#kfold = KFold(n_splits=6, random_state=0)

In [6]:
#results = cross_validate(rf_classifier,x,y,cv=kfold, scoring=scoring,return_train_score=False)


#  SVM Model

In [7]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y,test_size = 0.25, random_state = 42)

In [8]:
from sklearn import svm
svm_classifier=svm.SVC(random_state = 42)

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(svm_classifier.get_params())





Parameters currently in use:

{'C': 1.0,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'auto_deprecated',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 42,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}


In [9]:
from sklearn.model_selection import RandomizedSearchCV

#criterio
kernel = ['rbf','sigmoid','linear']
gamma = [0.1, 0.01, 0.001]
C = [0.001, 0.01, 0.1, 10, 25, 50, 100]


# Create the random grid
random_grid = {'kernel':kernel,'gamma':gamma,'C':C}
               

pprint(random_grid)


{'C': [0.001, 0.01, 0.1, 10, 25, 50, 100],
 'gamma': [0.1, 0.01, 0.001],
 'kernel': ['rbf', 'sigmoid', 'linear']}


In [10]:

# Use the random grid to search for best hyperparameters
# First create the base model to tune
svmm=svm.SVC()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
svm_random = RandomizedSearchCV(estimator=svmm, param_distributions=random_grid,
                              n_iter = 40, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
svm_random.fit(train_x, train_y)

Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 166.8min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
          fit_params=None, iid='warn', n_iter=40, n_jobs=-1,
          param_distributions={'kernel': ['rbf', 'sigmoid', 'linear'], 'C': [0.001, 0.01, 0.1, 10, 25, 50, 100], 'gamma': [0.1, 0.01, 0.001]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring='neg_mean_absolute_error',
          verbose=2)

In [11]:
svm_random.best_params_ 


#svm_random.fit(train_x, train_y)

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [12]:
print(svm_random.best_params_)
#print(rf_random.best_estimator_ )

{'kernel': 'rbf', 'C': 10, 'gamma': 0.001}


In [13]:
best_random = svm_random.best_estimator_
print(best_random)
y_predict = best_random.predict(test_x)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [14]:
from sklearn.metrics import accuracy_score
accura= accuracy_score(test_y, y_predict)  

from sklearn.metrics import f1_score
f1score= f1_score(test_y, y_predict)  


In [15]:
print("accuracy: ",accura)
print("f1_score: ",f1score)

('accuracy: ', 0.99955057442206685)
('f1_score: ', 0.84466019417475735)
